In [16]:
import pickle

with open("data/epl_2015.pkl", "rb") as f:
    loaded_list = pickle.load(f)

print(len(loaded_list))
# print(loaded_list[0])
loaded_list[0]["events"]

380


game_id  \
league             season game                                          
ENG-Premier League 1516   2015-08-08 Bournemouth-Aston Villa   958427   
                          2015-08-08 Bournemouth-Aston Villa   958427   
                          2015-08-08 Bournemouth-Aston Villa   958427   
                          2015-08-08 Bournemouth-Aston Villa   958427   
                          2015-08-08 Bournemouth-Aston Villa   958427   
...                                                               ...   
                          2015-08-08 Bournemouth-Aston Villa   958427   
                          2015-08-08 Bournemouth-Aston Villa   958427   
                          2015-08-08 Bournemouth-Aston Villa   958427   
                          2015-08-08 Bournemouth-Aston Villa   958427   
                          2015-08-08 Bournemouth-Aston Villa   958427   

                                                                  period  \
league             season game                                             
ENG-Premier League 1516   2015-08-08 Bournemouth-Aston Villa   FirstHalf   
                          2015-08-08 Bournemouth-Aston Villa   FirstHalf   
                          2015-08-08 Bournemouth-Aston Villa   FirstHalf   
                          2015-08-08 Bournemouth-Aston Villa   FirstHalf   
                          2015-08-08 Bournemouth-Aston Villa   FirstHalf   
...                                                                  ...   
                          2015-08-08 Bournemouth-Aston Villa  SecondHalf   
                          2015-08-08 Bournemouth-Aston Villa    PostGame   
                          2015-08-08 Bournemouth-Aston Villa    PostGame   
                          2015-08-08 Bournemouth-Aston Villa    PreMatch   
                          2015-08-08 Bournemouth-Aston Villa    PreMatch   

                                                              minute  second  \
league             season game                                                 
ENG-Premier League 1516   2015-08-08 Bournemouth-Aston Villa       0       0   
                          2015-08-08 Bournemouth-Aston Villa       0       0   
                          2015-08-08 Bournemouth-Aston Villa       0       0   
                          2015-08-08 Bournemouth-Aston Villa       0       1   
                          2015-08-08 Bournemouth-Aston Villa       0       3   
...                                                              ...     ...   
                          2015-08-08 Bournemouth-Aston Villa      95      23   
                          2015-08-08 Bournemouth-Aston Villa       0       0   
                          2015-08-08 Bournemouth-Aston Villa       0       0   
                          2015-08-08 Bournemouth-Aston Villa       0       0   
                          2015-08-08 Bournemouth-Aston Villa       0       0   

                                                              expanded_minute  \
league             season game                                                  
ENG-Premier League 1516   2015-08-08 Bournemouth-Aston Villa                0   
                          2015-08-08 Bournemouth-Aston Villa                0   
                          2015-08-08 Bournemouth-Aston Villa                0   
                          2015-08-08 Bournemouth-Aston Villa                0   
                          2015-08-08 Bournemouth-Aston Villa                0   
...                                                                       ...   
                          2015-08-08 Bournemouth-Aston Villa               97   
                          2015-08-08 Bournemouth-Aston Villa                7   
                          2015-08-08 Bournemouth-Aston Villa                7   
                          2015-08-08 Bournemouth-Aston Villa                0   
                          2015-08-08 Bournemouth-Aston Villa                0   

                            